<a href="https://colab.research.google.com/github/F-C0/Colab-Browser-with-Playwright/blob/main/Colab_Browser_with_Playwright.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Browser with Playwright

This notebook provides an **interactive browser interface inside Google Colab**, built on top of [Playwright](https://playwright.dev/).  
It lets you navigate the web, manage tabs, interact with pages, and track downloads — all without leaving a notebook.


## Features

- Multiple tabs with history (back/forward navigation).
- Click support (left, right, middle, double click) with modifier keys.
- Scroll and smooth wheel scrolling.
- Zoom in/out and reset (CSS zoom).
- Toggle between desktop and mobile view.
- Text input and keyboard key presses (Enter, Tab, Esc, etc.).
- Stop/reload navigation and home shortcut.
- Download manager with progress, ETA, speed, and logs.
- Screenshot-based rendering inside the notebook with clickable UI.

## How It Works

- Uses **Playwright (Chromium)** in headless mode to control browser pages.
- Injects a **custom JavaScript UI** in the Colab output cell for navigation, tabs, and interactions.
- Screenshots of the active page are streamed back to the notebook, enabling click and scroll mapping.
- State (tabs, active page, zoom, history) is managed in Python, while the front-end UI syncs via Colab’s `output.register_callback`.

## Limitations

- Rendering is screenshot-based, not a live DOM view.
- Performance depends on Colab runtime resources.
- Certain interactive or GPU-heavy sites may not work as expected.

In [ ]:
!pip install playwright pillow nest_asyncio
!playwright install chromium

In [6]:
# @title Start Colab Browser with Playwright
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import IPython
from google.colab import output
from PIL import Image
import io
import base64
import uuid
import json
import time
import os
from typing import List, Dict, Optional

# Enable nested event loops for Colab
nest_asyncio.apply()

DOWNLOAD_DIR = "/content/downloads"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)


def _fmt_bytes(n: Optional[int]) -> str:
    if n is None:
        return "?"
    for unit in ["B", "KB", "MB", "GB", "TB"]:
        if n < 1024.0:
            return f"{n:.1f} {unit}"
        n /= 1024.0
    return f"{n:.1f} PB"


def _fmt_speed(bps: Optional[float]) -> str:
    if not bps:
        return "-"
    return f"{_fmt_bytes(bps)}/s"


def _fmt_eta(remaining_bytes: Optional[int], bps: Optional[float]) -> str:
    if not bps or not remaining_bytes:
        return "-"
    secs = max(0, int(remaining_bytes / bps))
    if secs < 60:
        return f"{secs}s"
    mins, s = divmod(secs, 60)
    if mins < 60:
        return f"{mins}m {s}s"
    hrs, mins = divmod(mins, 60)
    return f"{hrs}h {mins}m"


class TabState:
    def __init__(self, page):
        self.page = page
        self.history: List[str] = []
        self.history_index: int = -1


class ColabBrowser:
    def __init__(self):
        self.browser = None
        self.playwright = None
        self.context = None
        self.tabs: List[TabState] = []
        self.active_tab: int = 0
        self.current_screenshot = None
        self.interface_id = str(uuid.uuid4())
        self.viewport_size = {"width": 1280, "height": 720}
        self.is_mobile = False
        # downloads state: id -> info
        self.downloads: Dict[str, Dict] = {}
        # zoom (CSS zoom)
        self.zoom = 1.0

    # ---------- Helpers ----------
    def _active_page(self):
        return self.tabs[self.active_tab].page if self.tabs else None

    async def _attach_listeners(self, page):
        async def on_frame_nav(frame):
            if frame == page.main_frame:
                url = frame.url
                tab = self.tabs[self.active_tab]
                # trim forward history then push
                if tab.history_index < len(tab.history) - 1:
                    tab.history = tab.history[: tab.history_index + 1]
                if not tab.history or tab.history[-1] != url:
                    tab.history.append(url)
                    tab.history_index = len(tab.history) - 1
        page.on("framenavigated", on_frame_nav)
        page.on("download", self._handle_download)

    async def init_browser(self):
        """Initialize the Playwright browser"""
        if self.browser is None:
            self.playwright = await async_playwright().start()
            self.browser = await self.playwright.chromium.launch(
                headless=True,
                args=["--no-sandbox", "--disable-dev-shm-usage"],
            )
            self.context = await self.browser.new_context(
                viewport=self.viewport_size,
                accept_downloads=True,
                user_agent=
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
            )
            # First tab
            page = await self.context.new_page()
            await self._attach_listeners(page)
            self.tabs = [TabState(page)]
            self.active_tab = 0
            await page.goto("https://www.google.com")
            self.tabs[0].history = [page.url]
            self.tabs[0].history_index = 0
        return self.browser, self._active_page()

    async def _handle_download(self, download):
        """Track file downloads; try to show progress by streaming when possible."""
        try:
            os.makedirs(DOWNLOAD_DIR, exist_ok=True)
            did = str(uuid.uuid4())
            filename = download.suggested_filename
            url = download.url
            start_ts = time.time()
            info = {
                "id": did,
                "filename": filename,
                "url": url,
                "status": "in_progress",
                "started_at": start_ts,
                "bytes_downloaded": 0,
                "total_bytes": None,
                "speed_bps": None,
                "path": os.path.join(DOWNLOAD_DIR, filename),
                "temp_path": os.path.join(DOWNLOAD_DIR, f".{filename}.part"),
                "via": "stream"  # or 'playwright'
            }
            self.downloads[did] = info

            async def stream_download():
                nonlocal info
                # Try to stream via aiohttp with cookies
                try:
                    import aiohttp
                except ImportError:
                    info["via"] = "playwright"
                    await download.save_as(info["path"])  # fallback, no progress
                    info.update({
                        "status": "completed",
                        "bytes_downloaded": os.path.getsize(info["path"]) if os.path.exists(info["path"]) else None,
                        "total_bytes": os.path.getsize(info["path"]) if os.path.exists(info["path"]) else None,
                        "speed_bps": None,
                    })
                    return

                # Build cookie header for domain
                cookies = await self.context.cookies()
                cookie_header = "; ".join(
                    [f"{c['name']}={c['value']}" for c in cookies]
                )
                headers = {
                    "User-Agent": await self._active_page().evaluate("() => navigator.userAgent"),
                    "Referer": self._active_page().url,
                }
                if cookie_header:
                    headers["Cookie"] = cookie_header

                chunk = 64 * 1024
                last_ts = time.time()
                last_bytes = 0

                try:
                    timeout = aiohttp.ClientTimeout(total=None, sock_read=600)
                    async with aiohttp.ClientSession(timeout=timeout) as session:
                        # HEAD to get size (best effort)
                        try:
                            async with session.head(url, headers=headers, allow_redirects=True) as resp_head:
                                cl = resp_head.headers.get("Content-Length")
                                if cl and cl.isdigit():
                                    info["total_bytes"] = int(cl)
                        except Exception:
                            pass
                        # GET streaming
                        async with session.get(url, headers=headers, allow_redirects=True) as resp:
                            resp.raise_for_status()
                            if not info.get("total_bytes"):
                                cl = resp.headers.get("Content-Length")
                                if cl and cl.isdigit():
                                    info["total_bytes"] = int(cl)
                            written = 0
                            with open(info["temp_path"], "wb") as f:
                                async for part in resp.content.iter_chunked(chunk):
                                    if part:
                                        f.write(part)
                                        written += len(part)
                                        info["bytes_downloaded"] = written
                                        # speed
                                        now = time.time()
                                        dt = now - last_ts
                                        if dt >= 0.5:
                                            delta = written - last_bytes
                                            info["speed_bps"] = delta / dt
                                            last_ts = now
                                            last_bytes = written
                except Exception:
                    # Fallback to Playwright managed
                    info["via"] = "playwright"
                    await download.save_as(info["path"])  # still saves to final
                    info.update({
                        "status": "completed",
                        "bytes_downloaded": os.path.getsize(info["path"]) if os.path.exists(info["path"]) else None,
                        "total_bytes": os.path.getsize(info["path"]) if os.path.exists(info["path"]) else None,
                        "speed_bps": None,
                    })
                    return

                # finalize
                try:
                    os.replace(info["temp_path"], info["path"])
                except FileNotFoundError:
                    pass
                info["status"] = "completed"

            # run background
            asyncio.create_task(stream_download())
        except Exception:
            pass

    async def _update_history(self, url):
        tab = self.tabs[self.active_tab]
        if tab.history_index < len(tab.history) - 1:
            tab.history = tab.history[: tab.history_index + 1]
        tab.history.append(url)
        tab.history_index = len(tab.history) - 1

    async def take_screenshot(self):
        page = self._active_page()
        if page is None:
            await self.init_browser()
            page = self._active_page()
        screenshot_bytes = await page.screenshot(full_page=False)
        self.current_screenshot = Image.open(io.BytesIO(screenshot_bytes))
        buffered = io.BytesIO()
        self.current_screenshot.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        return img_str

    async def navigate_to_url(self, url):
        page = self._active_page()
        if page is None:
            await self.init_browser()
        try:
            if not url.startswith(("http://", "https://")):
                url = "https://" + url
            await self._active_page().goto(url, wait_until="domcontentloaded")
            await self._update_history(self._active_page().url)
            screenshot = await self.take_screenshot()
            return await self._compose_state(screenshot=screenshot)
        except Exception as e:
            return {"error": str(e)}

    async def go_back(self):
        tab = self.tabs[self.active_tab]
        if tab.history_index > 0:
            tab.history_index -= 1
            url = tab.history[tab.history_index]
            await self._active_page().goto(url)
            screenshot = await self.take_screenshot()
            return await self._compose_state(screenshot=screenshot)
        return await self.get_page_info()

    async def go_forward(self):
        tab = self.tabs[self.active_tab]
        if tab.history_index < len(tab.history) - 1:
            tab.history_index += 1
            url = tab.history[tab.history_index]
            await self._active_page().goto(url)
            screenshot = await self.take_screenshot()
            return await self._compose_state(screenshot=screenshot)
        return await self.get_page_info()

    async def refresh_page(self):
        page = self._active_page()
        if page is None:
            return {"error": "Browser not initialized"}
        await page.reload()
        screenshot = await self.take_screenshot()
        return await self._compose_state(screenshot=screenshot)

    async def stop_loading(self):
        page = self._active_page()
        if page is None:
            return {"error": "Browser not initialized"}
        try:
            await page.evaluate("() => window.stop()")
        except Exception:
            pass
        screenshot = await self.take_screenshot()
        return await self._compose_state(screenshot=screenshot)

    async def handle_click(self, x, y, button="left", modifiers=None, click_count=1):
        page = self._active_page()
        if page is None or self.current_screenshot is None:
            return {"error": "Browser not initialized"}
        click_x = x * self.viewport_size["width"]
        click_y = y * self.viewport_size["height"]
        modifiers = modifiers or []
        try:
            # Decide if open in new tab desired
            open_in_new_tab = button == "middle" or ("Control" in modifiers)
            href = None
            try:
                href = await page.evaluate(
                    "([x,y]) => { const el = document.elementFromPoint(x,y); const a = el ? el.closest('a') : null; return a ? a.href : null; }",
                    [click_x, click_y],
                )
            except Exception:
                href = None

            if open_in_new_tab and href:
                await self.new_tab(href)
            else:
                await page.locator("html").click(
                    position={"x": click_x, "y": click_y},
                    button=button,
                    click_count=click_count,
                    modifiers=modifiers,
                )
                await asyncio.sleep(0.5)
            screenshot = await self.take_screenshot()
            return await self._compose_state(screenshot=screenshot)
        except Exception as e:
            return {"error": str(e)}

    async def scroll_page(self, direction, amount=300):
        page = self._active_page()
        if page is None:
            return {"error": "Browser not initialized"}
        try:
            dy = 0
            if direction == "up":
                dy = -amount
            elif direction == "down":
                dy = amount
            elif direction == "left":
                await page.mouse.wheel(-amount, 0)
            elif direction == "right":
                await page.mouse.wheel(amount, 0)
            if dy != 0:
                await page.mouse.wheel(0, dy)
            await asyncio.sleep(0.2)
            screenshot = await self.take_screenshot()
            return {"imagen": screenshot}
        except Exception as e:
            return {"error": str(e)}

    async def wheel_scroll(self, delta_x, delta_y):
        page = self._active_page()
        if page is None:
            return {"error": "Browser not initialized"}
        try:
            await page.mouse.wheel(delta_x, delta_y)
            await asyncio.sleep(0.05)
            screenshot = await self.take_screenshot()
            return {"imagen": screenshot}
        except Exception as e:
            return {"error": str(e)}

    async def type_text(self, text):
        page = self._active_page()
        if page is None:
            return {"error": "Browser not initialized"}
        try:
            await page.keyboard.type(text)
            await asyncio.sleep(0.2)
            screenshot = await self.take_screenshot()
            return {"imagen": screenshot}
        except Exception as e:
            return {"error": str(e)}

    async def send_key(self, key):
        page = self._active_page()
        if page is None:
            return {"error": "Browser not initialized"}
        try:
            await page.keyboard.press(key)
            await asyncio.sleep(0.2)
            screenshot = await self.take_screenshot()
            return {"imagen": screenshot}
        except Exception as e:
            return {"error": str(e)}

    async def toggle_mobile_view(self):
        self.is_mobile = not self.is_mobile
        if self.is_mobile:
            viewport = {"width": 375, "height": 667}
            user_agent = "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X)"
        else:
            viewport = {"width": 1280, "height": 720}
            user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
        self.viewport_size = viewport
        await self._active_page().set_viewport_size(viewport)
        try:
            await self.context.set_extra_http_headers({"User-Agent": user_agent})
        except Exception:
            pass
        screenshot = await self.take_screenshot()
        return {"imagen": screenshot, "is_mobile": self.is_mobile}

    async def set_zoom(self, action: str):
        if action == "+":
            self.zoom = min(2.0, self.zoom + 0.1)
        elif action == "-":
            self.zoom = max(0.5, self.zoom - 0.1)
        else:
            self.zoom = 1.0
        try:
            await self._active_page().evaluate(
                "z => { document.body.style.zoom = z; }", self.zoom
            )
        except Exception:
            pass
        screenshot = await self.take_screenshot()
        return await self._compose_state(screenshot=screenshot)

    # ---------- Tabs ----------
    async def new_tab(self, url: Optional[str] = None):
        if self.browser is None:
            await self.init_browser()
        page = await self.context.new_page()
        await self._attach_listeners(page)
        self.tabs.append(TabState(page))
        self.active_tab = len(self.tabs) - 1
        if url is None:
            url = "https://www.google.com"
        await page.goto(url)
        self.tabs[self.active_tab].history = [page.url]
        self.tabs[self.active_tab].history_index = 0
        screenshot = await self.take_screenshot()
        return await self._compose_state(screenshot=screenshot)

    async def switch_tab(self, index: int):
        if 0 <= index < len(self.tabs):
            self.active_tab = index
        screenshot = await self.take_screenshot()
        return await self._compose_state(screenshot=screenshot)

    async def close_tab(self, index: int):
        if len(self.tabs) <= 1:
            return await self.get_page_info()
        if 0 <= index < len(self.tabs):
            page = self.tabs[index].page
            try:
                await page.close()
            except Exception:
                pass
            del self.tabs[index]
            if self.active_tab >= len(self.tabs):
                self.active_tab = len(self.tabs) - 1
        screenshot = await self.take_screenshot()
        return await self._compose_state(screenshot=screenshot)

    # ---------- Downloads status ----------
    async def get_downloads(self):
        items = []
        for d in self.downloads.values():
            total = d.get("total_bytes")
            done = d.get("bytes_downloaded") or 0
            speed = d.get("speed_bps")
            remaining = (total - done) if (total and done is not None) else None
            items.append({
                "id": d["id"],
                "filename": d["filename"],
                "status": d["status"],
                "url": d["url"],
                "bytes_downloaded": done,
                "total_bytes": total,
                "progress": (done / total) if total else None,
                "speed_bps": speed,
                "speed_h": _fmt_speed(speed),
                "eta": _fmt_eta(remaining, speed),
                "done_h": _fmt_bytes(done),
                "total_h": _fmt_bytes(total) if total else "?",
                "via": d.get("via"),
            })
        return {"downloads": items}

    # ---------- Page info ----------
    async def _compose_state(self, screenshot: Optional[str] = None):
        page = self._active_page()
        title = await page.title()
        tabs_meta = []
        for t in self.tabs:
            try:
                tabs_meta.append({"title": await t.page.title(), "url": t.page.url})
            except Exception:
                tabs_meta.append({"title": "Tab", "url": ""})
        data = {
            "imagen": screenshot or await self.take_screenshot(),
            "url": page.url,
            "title": title,
            "can_go_back": self.tabs[self.active_tab].history_index > 0,
            "can_go_forward": self.tabs[self.active_tab].history_index < len(self.tabs[self.active_tab].history) - 1,
            "is_mobile": self.is_mobile,
            "tabs": tabs_meta,
            "active_tab": self.active_tab,
            "zoom": self.zoom,
        }
        return data

    async def get_page_info(self):
        if not self.tabs:
            await self.init_browser()
        return await self._compose_state()

    # ---------- Element info (for future context menu) ----------
    async def get_element_info(self, x, y):
        page = self._active_page()
        click_x = x * self.viewport_size["width"]
        click_y = y * self.viewport_size["height"]
        try:
            info = await page.evaluate(
                "([x,y]) => {\n                    const el = document.elementFromPoint(x,y);\n                    if (!el) return null;\n                    const a = el.closest('a');\n                    return {\n                      tag: el.tagName,\n                      text: (el.innerText || '').slice(0,200),\n                      href: a ? a.href : null\n                    };\n                }",
                [click_x, click_y],
            )
            return {"element": info}
        except Exception as e:
            return {"error": str(e)}

    # ---------- UI injection ----------
    def create_interface(self):
        js_code = f"""
        // Remove existing interface if any
        const existingContainer = document.getElementById('browser-container-{self.interface_id}');
        if (existingContainer) {{ existingContainer.remove(); }}

        // Main container
        const container = document.createElement('div');
        container.id = 'browser-container-{self.interface_id}';
        container.style.cssText = `
            border: 2px solid #ddd;
            border-radius: 8px;
            margin: 10px 0;
            background: #f8f9fa;
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
        `;

        // Tabs bar
        const tabsBar = document.createElement('div');
        tabsBar.style.cssText = `
            display: flex; align-items: center; gap: 6px; padding: 6px; background: #f0f2f5; border-bottom: 1px solid #ddd;
        `;
        const tabsContainer = document.createElement('div');
        tabsContainer.style.cssText = `display:flex; gap:4px; flex-wrap:wrap;`;
        const newTabBtn = document.createElement('button');
        newTabBtn.textContent = '+ New Tab';
        newTabBtn.style.cssText = `padding:4px 8px; border:1px solid #ccc; background:white; border-radius:4px; cursor:pointer;`;
        tabsBar.appendChild(tabsContainer);
        tabsBar.appendChild(newTabBtn);

        // Browser toolbar
        const toolbar = document.createElement('div');
        toolbar.style.cssText = `
            display: flex; align-items: center; gap: 8px; padding: 8px; background: #e9ecef; border-bottom: 1px solid #ddd; border-radius: 6px 6px 0 0;
        `;
        const mkBtn = (label, title) => {{
            const b = document.createElement('button'); b.textContent = label; b.title = title; b.style.cssText = `padding:6px 10px; border:1px solid #ccc; background:white; border-radius:4px; cursor:pointer; font-size:14px;`; return b;
        }};
        const backBtn = mkBtn('←','Back'); backBtn.id = 'back-btn-{self.interface_id}'; backBtn.disabled = true;
        const forwardBtn = mkBtn('→','Forward'); forwardBtn.id = 'forward-btn-{self.interface_id}'; forwardBtn.disabled = true;
        const refreshBtn = mkBtn('↻','Refresh'); refreshBtn.id = 'refresh-btn-{self.interface_id}';
        const stopBtn = mkBtn('⛔','Stop loading'); stopBtn.id = 'stop-btn-{self.interface_id}';
        const homeBtn = mkBtn('🏠','Home'); homeBtn.id = 'home-btn-{self.interface_id}';
        const zoomOutBtn = mkBtn('−','Zoom out');
        const zoomResetBtn = mkBtn('⤿','Reset zoom');
        const zoomInBtn = mkBtn('+','Zoom in');
        const urlInput = document.createElement('input'); urlInput.type='text'; urlInput.placeholder='Enter URL...'; urlInput.style.cssText = `flex:1; padding:8px 12px; border:1px solid #ccc; border-radius:4px; font-size:14px;`;
        const mobileBtn = mkBtn('📱','Toggle Mobile View'); mobileBtn.id = 'mobile-btn-{self.interface_id}';

        toolbar.append(backBtn, forwardBtn, refreshBtn, stopBtn, homeBtn, urlInput, mobileBtn, zoomOutBtn, zoomResetBtn, zoomInBtn);

        // Control panel
        const controlPanel = document.createElement('div');
        controlPanel.style.cssText = `display:flex; align-items:center; gap:8px; padding:8px; background:#f1f3f4; border-bottom:1px solid #ddd;`;
        const scrollUp = mkBtn('↑','Scroll Up');
        const scrollDown = mkBtn('↓','Scroll Down');
        const scrollTop = mkBtn('⤒','Top');
        const scrollBottom = mkBtn('⤓','Bottom');
        const textInput = document.createElement('input'); textInput.type='text'; textInput.placeholder='Type text...'; textInput.style.cssText = `padding:6px 8px; border:1px solid #ccc; border-radius:4px; width:200px;`;
        const typeBtn = mkBtn('Type','Type text');
        const enterBtn = mkBtn('Enter','Enter');
        const tabBtn = mkBtn('Tab','Tab');
        const escBtn = mkBtn('Esc','Escape');
        controlPanel.append(scrollUp, scrollDown, scrollTop, scrollBottom, textInput, typeBtn, enterBtn, tabBtn, escBtn);

        // Status + downloads + logs
        const statusBar = document.createElement('div');
        statusBar.style.cssText = `padding:4px 8px; background:#e9ecef; border-top:1px solid #ddd; font-size:12px; color:#666; display:flex; justify-content:space-between; align-items:center; gap:10px;`;
        const statusText = document.createElement('div'); statusText.textContent = 'Ready';
        const dlToggle = mkBtn('⬇ Downloads','Show downloads');
        const logsToggle = mkBtn('🪵 Logs','Show logs');
        const clearLogsBtn = mkBtn('Clear','Clear logs');
        const btnsRight = document.createElement('div'); btnsRight.style.cssText = `display:flex; gap:6px;`;
        btnsRight.append(dlToggle, logsToggle, clearLogsBtn);
        statusBar.append(statusText, btnsRight);

        const dlPanel = document.createElement('div'); dlPanel.style.cssText = `display:none; padding:8px; background:#fff; border-top:1px solid #ddd; max-height:200px; overflow:auto; font-family:monospace;`;
        const logsPanel = document.createElement('div'); logsPanel.style.cssText = `display:none; padding:8px; background:#fff; border-top:1px solid #ddd; max-height:200px; overflow:auto; font-family:monospace;`;

        // Image container
        const imgContainer = document.createElement('div'); imgContainer.style.cssText = `background:white; overflow:auto; max-height:600px; position:relative;`;
        const img = document.createElement('img'); img.style.cssText = `max-width:100%; cursor:crosshair; display:block; user-select:none;`; img.draggable = false;
        imgContainer.appendChild(img);

        // Assemble
        container.appendChild(tabsBar);
        container.appendChild(toolbar);
        container.appendChild(controlPanel);
        container.appendChild(imgContainer);
        container.appendChild(dlPanel);
        container.appendChild(logsPanel);
        container.appendChild(statusBar);
        document.body.appendChild(container);

        // Tabs rendering
        function renderTabs(tabs, active) {{
            tabsContainer.innerHTML='';
            tabs.forEach((t, i) => {{
                const el = document.createElement('div');
                el.style.cssText = `display:flex; align-items:center; gap:6px; padding:4px 8px; border:1px solid #ccc; border-radius:4px; background:${{i===active?'#dbeafe':'white'}}; cursor:pointer;`;
                const ttl = document.createElement('span'); ttl.textContent = (t.title||'Tab'); ttl.style.maxWidth='180px'; ttl.style.overflow='hidden'; ttl.style.textOverflow='ellipsis'; ttl.style.whiteSpace='nowrap';
                const close = document.createElement('button'); close.textContent='×'; close.style.cssText = `border:1px solid #ccc; background:white; border-radius:3px; cursor:pointer;`;
                el.appendChild(ttl); el.appendChild(close);
                el.addEventListener('click', async (ev) => {{ if (ev.target===close) return; log('Switch tab to #' + (i+1)); const r = await google.colab.kernel.invokeFunction('browser_switch_tab_{self.interface_id}', [i], {{}}); updateInterface(r.data['application/json']); }});
                close.addEventListener('click', async (ev) => {{ ev.stopPropagation(); log('Close tab #' + (i+1)); const r = await google.colab.kernel.invokeFunction('browser_close_tab_{self.interface_id}', [i], {{}}); updateInterface(r.data['application/json']); }});
                tabsContainer.appendChild(el);
            }});
        }}

        newTabBtn.onclick = async () => {{
            log('New tab');
            const r = await google.colab.kernel.invokeFunction('browser_new_tab_{self.interface_id}', [], {{}});
            updateInterface(r.data['application/json']);
        }};

        // Logging utilities
        function log(msg, type='info') {{
            const row = document.createElement('div');
            row.textContent = '[' + new Date().toLocaleTimeString() + '] ' + msg;
            row.style.color = (type==='error') ? '#b00020' : '#333';
            logsPanel.appendChild(row);
            logsPanel.scrollTop = logsPanel.scrollHeight;
            try {{ console.log('NAV-LOG:', msg); }} catch (e) {{}}
        }}

        dlToggle.onclick = () => {{ dlPanel.style.display = dlPanel.style.display==='none'?'block':'none'; }};
        logsToggle.onclick = () => {{ logsPanel.style.display = logsPanel.style.display==='none'?'block':'none'; }};
        clearLogsBtn.onclick = () => {{ logsPanel.innerHTML=''; }};

        // Downloads UI
        function renderDownloads(list) {{
            dlPanel.innerHTML = list.length===0 ? '<em>No downloads</em>' : '';
            list.forEach(d => {{
                const row = document.createElement('div'); row.style.marginBottom='6px';
                const prog = (d.progress!=null) ? Math.round(d.progress*100) + '%' : '?';
                row.textContent = '[' + d.status + '] ' + d.filename + ' - ' + d.done_h + '/' + d.total_h + ' (' + prog + ') @ ' + d.speed_h + ' ETA ' + d.eta + ' (' + d.via + ')';
                dlPanel.appendChild(row);
            }});
        }}

        // Track download status changes
        const prevDlStatus = {{}};

        // Event handlers
        backBtn.onclick = async () => {{ log('Back'); const r = await google.colab.kernel.invokeFunction('browser_go_back_{self.interface_id}', [], {{}}); const d = r.data['application/json']; if (d.error) log('Back error: ' + d.error, 'error'); updateInterface(d); }};
        forwardBtn.onclick = async () => {{ log('Forward'); const r = await google.colab.kernel.invokeFunction('browser_go_forward_{self.interface_id}', [], {{}}); const d = r.data['application/json']; if (d.error) log('Forward error: ' + d.error, 'error'); updateInterface(d); }};
        refreshBtn.onclick = async () => {{ log('Refresh'); const r = await google.colab.kernel.invokeFunction('browser_refresh_{self.interface_id}', [], {{}}); const d = r.data['application/json']; if (d.error) log('Refresh error: ' + d.error, 'error'); updateInterface(d); }};
        stopBtn.onclick = async () => {{ log('Stop loading'); const r = await google.colab.kernel.invokeFunction('browser_stop_{self.interface_id}', [], {{}}); const d = r.data['application/json']; if (d.error) log('Stop error: ' + d.error, 'error'); updateInterface(d); }};
        homeBtn.onclick = async () => {{ urlInput.value = 'https://www.google.com'; log('Home'); const r = await google.colab.kernel.invokeFunction('browser_navigate_{self.interface_id}', [urlInput.value], {{}}); const d = r.data['application/json']; if (d.error) log('Navigate error: ' + d.error, 'error'); updateInterface(d); }};
        zoomInBtn.onclick = async () => {{ log('Zoom in'); const r = await google.colab.kernel.invokeFunction('browser_zoom_{self.interface_id}', ['+'], {{}}); updateInterface(r.data['application/json']); }};
        zoomOutBtn.onclick = async () => {{ log('Zoom out'); const r = await google.colab.kernel.invokeFunction('browser_zoom_{self.interface_id}', ['-'], {{}}); updateInterface(r.data['application/json']); }};
        zoomResetBtn.onclick = async () => {{ log('Zoom reset'); const r = await google.colab.kernel.invokeFunction('browser_zoom_{self.interface_id}', ['reset'], {{}}); updateInterface(r.data['application/json']); }};

        urlInput.onkeypress = async (e) => {{ if (e.key==='Enter') {{ log('Navigate: ' + urlInput.value); const r = await google.colab.kernel.invokeFunction('browser_navigate_{self.interface_id}', [urlInput.value], {{}}); const d = r.data['application/json']; if (d.error) log('Navigate error: ' + d.error, 'error'); updateInterface(d); }} }};
        mobileBtn.onclick = async () => {{ log('Toggle mobile'); const r = await google.colab.kernel.invokeFunction('browser_toggle_mobile_{self.interface_id}', [], {{}}); updateInterface(r.data['application/json']); }};

        // Scroll controls
        const wheel = (dx, dy) => google.colab.kernel.invokeFunction('browser_wheel_{self.interface_id}', [dx, dy], {{}}).then(r => r.data['application/json']).then(d => {{ updateInterface(d); }}).catch(err => log('Wheel error: ' + err, 'error'));
        scrollUp.onclick = async () => {{ log('Scroll up'); wheel(0, -300); }};
        scrollDown.onclick = async () => {{ log('Scroll down'); wheel(0, 300); }};
        scrollTop.onclick = async () => {{ log('Scroll top'); wheel(0, -99999); }};
        scrollBottom.onclick = async () => {{ log('Scroll bottom'); wheel(0, 99999); }};

        // Typing controls
        typeBtn.onclick = async () => {{
            if (!textInput.value) {{ log('Type: empty input', 'error'); return; }}
            log('Type ' + textInput.value.length + ' chars');
            const r = await google.colab.kernel.invokeFunction('browser_type_{self.interface_id}', [textInput.value], {{}});
            const d = r.data['application/json'];
            if (d && d.error) log('Type error: ' + d.error, 'error'); else log('Type OK');
            updateInterface(d);
            textInput.value = '';
        }};
        textInput.onkeypress = async (e) => {{ if (e.key==='Enter') typeBtn.click(); }};
        enterBtn.onclick = async () => {{ log('Key: Enter'); const r = await google.colab.kernel.invokeFunction('browser_key_{self.interface_id}', ['Enter'], {{}}); updateInterface(r.data['application/json']); }};
        tabBtn.onclick = async () => {{ log('Key: Tab'); const r = await google.colab.kernel.invokeFunction('browser_key_{self.interface_id}', ['Tab'], {{}}); updateInterface(r.data['application/json']); }};
        escBtn.onclick = async () => {{ log('Key: Escape'); const r = await google.colab.kernel.invokeFunction('browser_key_{self.interface_id}', ['Escape'], {{}}); updateInterface(r.data['application/json']); }};

        // Natural wheel scrolling on image
        imgContainer.addEventListener('wheel', async (e) => {{
            e.preventDefault();
            const r = await google.colab.kernel.invokeFunction('browser_wheel_{self.interface_id}', [e.deltaX, e.deltaY], {{}});
            updateInterface(r.data['application/json']);
        }}, {{ passive: false }});

        // Click handlers with modifiers and middle/double click
        function clickPayload(event) {{
            const rect = img.getBoundingClientRect();
            const x = (event.clientX - rect.left) / rect.width;
            const y = (event.clientY - rect.top) / rect.height;
            const mods = [];
            if (event.ctrlKey) mods.push('Control');
            if (event.shiftKey) mods.push('Shift');
            if (event.altKey) mods.push('Alt');
            if (event.metaKey) mods.push('Meta');
            let button = 'left';
            if (event.button===1) button='middle';
            if (event.button===2) button='right';
            return [x, y, button, mods, event.detail || 1];
        }}

        img.addEventListener('contextmenu', e => e.preventDefault());
        img.addEventListener('click', async (event) => {{
            const [x,y,button,mods,count] = clickPayload(event);
            log('Click ' + button + ' at ' + Math.round(x*100) + '%, ' + Math.round(y*100) + '%');
            const r = await google.colab.kernel.invokeFunction('browser_click_{self.interface_id}', [x,y,button,mods,count], {{}});
            const d = r.data['application/json']; if (d && d.error) log('Click error: ' + d.error, 'error'); updateInterface(d);
        }});
        img.addEventListener('auxclick', async (event) => {{
            const [x,y,button,mods,count] = clickPayload(event);
            log('AuxClick ' + button + ' at ' + Math.round(x*100) + '%, ' + Math.round(y*100) + '%');
            const r = await google.colab.kernel.invokeFunction('browser_click_{self.interface_id}', [x,y,button,mods,count], {{}});
            const d = r.data['application/json']; if (d && d.error) log('AuxClick error: ' + d.error, 'error'); updateInterface(d);
        }});
        img.addEventListener('dblclick', async (event) => {{
            const [x,y,button,mods,_] = clickPayload(event);
            log('DoubleClick at ' + Math.round(x*100) + '%, ' + Math.round(y*100) + '%');
            const r = await google.colab.kernel.invokeFunction('browser_click_{self.interface_id}', [x,y,button,mods,2], {{}});
            const d = r.data['application/json']; if (d && d.error) log('DoubleClick error: ' + d.error, 'error'); updateInterface(d);
        }});

        // Update interface
        function updateInterface(data) {{
            if (!data) return;
            if (data.error) {{ statusText.textContent = 'Error: ' + data.error; statusText.style.color='red'; log('Backend error: ' + data.error, 'error'); return; }}
            if (data.imagen) {{ img.src = 'data:image/png;base64,' + data.imagen; }}
            if (data.url) {{ urlInput.value = data.url; statusText.textContent = (data.title||'') + ' - ' + data.url; statusText.style.color='#666'; }}
            if (typeof data.can_go_back !== 'undefined') backBtn.disabled = !data.can_go_back;
            if (typeof data.can_go_forward !== 'undefined') forwardBtn.disabled = !data.can_go_forward;
            if (data.is_mobile !== undefined) {{ mobileBtn.style.background = data.is_mobile ? '#007bff' : 'white'; mobileBtn.style.color = data.is_mobile ? 'white' : 'black'; }}
            if (data.tabs) {{ renderTabs(data.tabs, data.active_tab||0); }}
        }}

        // Poll downloads + log on state changes
        let dlTimer = setInterval(async () => {{
            try {{
                const r = await google.colab.kernel.invokeFunction('browser_get_downloads_{self.interface_id}', [], {{}});
                const d = r.data['application/json'];
                if (d && d.downloads) {{
                    renderDownloads(d.downloads);
                    d.downloads.forEach(item => {{
                        const prev = prevDlStatus[item.id];
                        if (!prev) {{
                            prevDlStatus[item.id] = item.status;
                            log('Download started: ' + item.filename);
                        }} else if (prev !== item.status) {{
                            prevDlStatus[item.id] = item.status;
                            log('Download ' + item.status + ': ' + item.filename, item.status==='completed' ? 'info' : 'error');
                        }}
                    }});
                }}
            }} catch (e) {{ log('Downloads poll error: ' + e, 'error'); }}
        }}, 1000);

        // Initial load
        google.colab.kernel.invokeFunction('browser_init_{self.interface_id}', [], {{}}).then(r => updateInterface(r.data['application/json']));
        """
        IPython.display.display(IPython.display.Javascript(js_code))

# Create browser instance
browser_instance = ColabBrowser()

# Register all callback functions
def register_callbacks():
    """Register all browser callback functions"""

    def sync_wrapper(async_func):
        def wrapper(*args, **kwargs):
            try:
                loop = asyncio.get_event_loop()
                return IPython.display.JSON(loop.run_until_complete(async_func(*args, **kwargs)))
            except Exception as e:
                return IPython.display.JSON({"error": str(e)})
        return wrapper

    # Navigation and info
    output.register_callback(f'browser_init_{browser_instance.interface_id}', sync_wrapper(browser_instance.get_page_info))
    output.register_callback(f'browser_navigate_{browser_instance.interface_id}', sync_wrapper(browser_instance.navigate_to_url))
    output.register_callback(f'browser_go_back_{browser_instance.interface_id}', sync_wrapper(browser_instance.go_back))
    output.register_callback(f'browser_go_forward_{browser_instance.interface_id}', sync_wrapper(browser_instance.go_forward))
    output.register_callback(f'browser_refresh_{browser_instance.interface_id}', sync_wrapper(browser_instance.refresh_page))
    output.register_callback(f'browser_stop_{browser_instance.interface_id}', sync_wrapper(browser_instance.stop_loading))

    # Tabs
    output.register_callback(f'browser_new_tab_{browser_instance.interface_id}', sync_wrapper(browser_instance.new_tab))
    output.register_callback(f'browser_switch_tab_{browser_instance.interface_id}', sync_wrapper(browser_instance.switch_tab))
    output.register_callback(f'browser_close_tab_{browser_instance.interface_id}', sync_wrapper(browser_instance.close_tab))

    # Interaction
    output.register_callback(f'browser_click_{browser_instance.interface_id}', sync_wrapper(browser_instance.handle_click))
    output.register_callback(f'browser_scroll_{browser_instance.interface_id}', sync_wrapper(browser_instance.scroll_page))
    output.register_callback(f'browser_wheel_{browser_instance.interface_id}', sync_wrapper(browser_instance.wheel_scroll))
    output.register_callback(f'browser_type_{browser_instance.interface_id}', sync_wrapper(browser_instance.type_text))
    output.register_callback(f'browser_key_{browser_instance.interface_id}', sync_wrapper(browser_instance.send_key))
    output.register_callback(f'browser_toggle_mobile_{browser_instance.interface_id}', sync_wrapper(browser_instance.toggle_mobile_view))
    output.register_callback(f'browser_zoom_{browser_instance.interface_id}', sync_wrapper(browser_instance.set_zoom))
    output.register_callback(f'browser_element_info_{browser_instance.interface_id}', sync_wrapper(browser_instance.get_element_info))

    # Downloads
    output.register_callback(f'browser_get_downloads_{browser_instance.interface_id}', sync_wrapper(browser_instance.get_downloads))

# Register callbacks and create interface
register_callbacks()
browser_instance.create_interface()

<IPython.core.display.Javascript object>